# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [54]:
import pandas as pd
import os
import numpy as np

In [55]:
#DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.
DATA_FOLDER = "./Data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Strategy

We will import and wrangle the data for each country separately and then combine it into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

In [56]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

### Guinea

In [57]:
dfs = {}
tables_with_duplicates = []
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Description', 'Totals'])
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable): # Check for duplicates
        tables_with_duplicates.append(file)
    dfs[file] = df

In [58]:
tables_with_duplicates

[]

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [59]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [60]:
dfs[0]

Variable,Cumulative (confirmed + probable + suspects),Fatality rate for confirmed and probables,New admits to CTE so far,New cases of confirmed,New cases of confirmed among health workers,New cases of probables,New cases of suspects,New contacts registered so far,New deaths registered,New deaths registered among health workers,...,Total deaths of confirmed,Total deaths of probables,Total deaths of suspects,Total deaths registered among health workers,Total new cases registered so far,Total number of admissions to CTE,Total number of exits from CTE,Total number of hospitalized cases in CTE,Total samples tested,Total suspected non-class cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-19,974,65%,8,12,0,1,3,182,5,0,...,463,162,5,35,16,1379,17,82,2505,838


In [61]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date

In [62]:
guinea.head(n=10)

Variable,Cumulative (confirmed + probable + suspects),Fatality rate for confirmed and probables,New admits to CTE so far,New cases of confirmed,New cases of confirmed among health workers,New cases of probables,New cases of suspects,New contacts registered so far,New deaths registered,New deaths registered among health workers,...,Total new cases registered so far,Total number of admissions to CTE,Total number of exits from CTE,Total number of female cases,Total number of hospitalized cases in CTE,Total number of male cases,Total of cured in confirmed cases in CTE,Total of deaths in confirmed cases in CTE,Total samples tested,Total suspected non-class cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-08-04,495,NaN,NaN,4,NaN,0,5,NaN,NaN,NaN,...,9,NaN,NaN,277.0,NaN,220.0,89.0,138.0,NaN,NaN
2014-08-26,661,66%,22,10,0,NaN,18,137,5,0,...,28,997,6,NaN,87,NaN,NaN,NaN,1700,518
2014-08-27,663,66%,12,10,0,0,12,127,2,0,...,22,999,14,NaN,80,NaN,NaN,NaN,1719,518
2014-08-30,707,66%,9,9,0,0,15,21,5,0,...,24,1094,33,NaN,82,NaN,NaN,NaN,1858,560
2014-08-31,749,65%,26,29,0,8,9,0,3,0,...,46,1113,10,NaN,99,NaN,NaN,NaN,1941,642
2014-09-02,790,63%,29,14,0,0,11,0,5,0,...,25,1144,24,NaN,105,NaN,NaN,NaN,2018,642
2014-09-04,823,61%,13,17,0,0,13,21,5,0,...,30,1191,15,NaN,103,NaN,NaN,NaN,249,682
2014-09-07,861,65%,50,11,2,0,5,5,4,0,...,16,1256,8,NaN,102,NaN,NaN,NaN,2236,714
2014-09-08,861,65%,50,11,2,0,5,5,4,0,...,16,1256,8,NaN,102,NaN,NaN,NaN,2236,714


### Liberia

In [63]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Variable', 'National'])
    df.Variable = df.Variable.astype('category')
        
    if len(df.Variable.unique()) != len(df.Variable): # Check for duplicates
        tables_with_duplicates.append(file)
            
    dfs[file] = df

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [64]:
tables_with_duplicates

['2014-10-04-v142.csv']

In [65]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
8,2014-10-04,Cumulative (confirmed + probable + suspected),3921.0
10,2014-10-04,Cumulative cases among HCW,190.0
12,2014-10-04,Cumulative deaths among HCW,92.0
17,2014-10-04,Total death/s in confirmed cases,1012.0
18,2014-10-04,Total death/s in probable cases,699.0
19,2014-10-04,Total death/s in suspected cases,488.0
20,2014-10-04,"Total death/s in confirmed, probable, suspecte...",2199.0
31,2014-10-04,Cumulative (confirmed + probable + suspected),3929.0
32,2014-10-04,Cumulative cases among HCW,192.0
33,2014-10-04,Cumulative deaths among HCW,94.0


7 Variables are repeated twice.

In [66]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [67]:
second - first

,National
Variable,
Cumulative (confirmed + probable + suspected),8.0
Cumulative cases among HCW,2.0
Cumulative deaths among HCW,2.0
Total death/s in confirmed cases,6.0
Total death/s in probable cases,2.0
Total death/s in suspected cases,3.0
"Total death/s in confirmed, probable, suspected cases",11.0


We decided to keep the bigger values as the difference is not very relevant.

In [68]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [69]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [70]:
dfs[0]

Variable,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Contacts lost to follow-up,Contacts seen,Contacts who completed 21 day follow-up,Cumulative admission/isolation,Cumulative cases among HCW,"Cumulative confirmed, probable and suspected cases",Cumulative deaths among HCW,Currently under follow-up,New Case/s (Probable),...,Total contacts listed,Total death/s in confirmed cases,"Total death/s in confirmed, probable, suspected cases",Total death/s in probable cases,Total death/s in suspected cases,Total discharges,Total no. currently in Treatment Units,Total probable cases,Total specimens tested,Total suspected cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-04,NaN,NaN,NaN,NaN,NaN,NaN,7721.0,NaN,NaN,1800.0,...,NaN,NaN,3205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
liberia = pd.concat(dfs).sort_index()

In [72]:
liberia.head(n=10)

Variable,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Contacts lost to follow-up,Contacts seen,Contacts who completed 21 day follow-up,Contacts who completed 21 day follow-up,Cumulative (confirmed + probable + suspected),Cumulative (confirmed + probable + suspects),Cumulative CFR,Cumulative admission/isolation,...,"Total death/s in confirmed, probable, suspected cases","Total death/s in confirmed, probable, suspected cases",Total death/s in probable cases,Total death/s in suspected cases,Total discharges,Total no. currently in Treatment Units,Total no. currently in Treatment Units,Total probable cases,Total specimens tested,Total suspected cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-16,NaN,NaN,NaN,95.0,NaN,11.0,NaN,NaN,NaN,5.0,...,NaN,16.0,6.0,2.0,3.0,NaN,5.0,6.0,28.0,4.0
2014-06-17,NaN,NaN,0.0,90.0,NaN,11.0,NaN,NaN,NaN,10.0,...,NaN,16.0,6.0,2.0,0.0,NaN,10.0,6.0,34.0,8.0
2014-06-22,NaN,25.0,0.0,232.0,NaN,17.0,NaN,NaN,NaN,15.0,...,NaN,25.0,8.0,1.0,0.0,NaN,15.0,8.0,49.0,6.0
2014-06-24,NaN,NaN,15.0,224.0,NaN,17.0,NaN,NaN,NaN,25.0,...,NaN,32.0,8.0,6.0,0.0,NaN,25.0,8.0,63.0,13.0
2014-06-25,NaN,NaN,15.0,112.0,NaN,17.0,NaN,NaN,NaN,25.0,...,NaN,37.0,8.0,9.0,0.0,NaN,25.0,9.0,4.0,17.0
2014-06-28,NaN,61.0,15.0,407.0,NaN,22.0,NaN,NaN,NaN,25.0,...,NaN,49.0,10.0,13.0,3.0,NaN,25.0,18.0,3.0,29.0
2014-06-29,NaN,NaN,15.0,0.0,NaN,22.0,NaN,NaN,NaN,25.0,...,NaN,49.0,10.0,13.0,1.0,NaN,25.0,18.0,4.0,29.0
2014-07-01,NaN,70.1,15.0,422.0,NaN,17.0,NaN,NaN,NaN,19.0,...,NaN,61.0,17.0,14.0,3.0,NaN,19.0,19.0,4.0,33.0
2014-07-02,NaN,74.3,15.0,423.0,NaN,40.0,NaN,NaN,NaN,18.0,...,NaN,66.0,20.0,14.0,0.0,NaN,18.0,22.0,0.0,34.0


### Sierra Leone

In [73]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path,
                     parse_dates=['date'],
                     usecols=['date', 'variable', 'National'])
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [74]:
tables_with_duplicates

['2014-11-10-v167.csv']

In [75]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
34,2014-11-10,pending,32.0
35,2014-11-10,positive_corpse,37.0
36,2014-11-10,negative_corpse,91.0
37,2014-11-10,pending,NaN
38,2014-11-10,positive_corpse,NaN
39,2014-11-10,negative_corpse,NaN


This time we decide to drop `NaN` values in favor of present data.

In [76]:
with_duplicates.drop_duplicates(subset='Variable', keep='first', inplace=True)

In [77]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [78]:
dfs[0]

Variable,cfr,contacts_followed,contacts_healthy,contacts_ill,contacts_not_seen,cum_completed_contacts,cum_confirmed,cum_contacts,cum_noncase,cum_probable,...,etc_new_deaths,etc_new_discharges,new_completed_contacts,new_confirmed,new_contacts,new_noncase,new_probable,new_suspected,percent_seen,population
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-02,34.8,3275,3221,14,40,3842,1146,7338,981,37,...,NaN,NaN,218,31,207,21,0,4,99%,6348350


In [79]:
sl = pd.concat(dfs).sort_index()

In [80]:
sl.head(n=10)

Variable,cfr,contacts_followed,contacts_healthy,contacts_ill,contacts_not_seen,cum_completed_contacts,cum_confirmed,cum_contacts,cum_noncase,cum_probable,...,new_positive,new_probable,new_samples,new_suspected,pending,percent_seen,population,positive_corpse,repeat_samples,total_lab_samples
Date,,,,,,,,,,,,,,,,,,,,,
2014-08-12,36.8,2255,1937,14,131,901,717,2838,657,37,...,NaN,1,NaN,10,NaN,87%,6348350,NaN,NaN,NaN
2014-08-13,37.2,NaN,1956,14,131,NaN,733,2099,680,38,...,NaN,1,NaN,3,NaN,94%,"6,348,350",NaN,NaN,NaN
2014-08-14,37.5,2268,2135,7,117,1852,747,4173,708,39,...,NaN,2,NaN,0,NaN,94%,"6,348,350",NaN,NaN,NaN
2014-08-15,37.9,2212,2137,7,95,1927,757,4280,726,37,...,NaN,1,NaN,6,NaN,97%,"6,348,350",NaN,NaN,NaN
2014-08-16,38.3,2323,2265,6,79,2021,775,4361,741,34,...,NaN,0,NaN,3,NaN,98%,"6,348,350",NaN,NaN,NaN
2014-08-17,39.2,2428,2311,8,97,2135,778,4451,758,37,...,NaN,0,NaN,1,NaN,96%,"6,348,350",NaN,NaN,NaN
2014-08-18,39.8,2510,2420,16,66,2243,783,4581,758,52,...,NaN,15,NaN,40,NaN,97%,"6,348,350",NaN,NaN,NaN
2014-08-19,39.8,2434,2347,14,71,2313,804,4686,802,40,...,NaN,3,NaN,16,NaN,97%,6348350,NaN,NaN,NaN
2014-08-20,39.6,2423,2227,11,172,2384,813,4804,817,37,...,NaN,0,NaN,1,NaN,92%,6348350,NaN,NaN,NaN


In [81]:
del dfs
del tables_with_duplicates

### Combining the data

In [82]:
guinea.columns

CategoricalIndex(['Cumulative (confirmed + probable + suspects)',
                  'Fatality rate for confirmed and probables',
                  'New admits to CTE so far', 'New cases of confirmed',
                  'New cases of confirmed among health workers',
                  'New cases of probables', 'New cases of suspects',
                  'New contacts registered so far', 'New deaths registered',
                  'New deaths registered among health workers',
                  'New deaths registered today',
                  'New deaths registered today (confirmed)',
                  'New deaths registered today (probables)',
                  'New deaths registered today (suspects)',
                  'Number of confirmed cases among health workers',
                  'Number of contacts followed today',
                  'Number of contacts followed yesterday',
                  'Number of contacts lost to follow up',
                  'Number of contacts out of the trac

In [83]:
liberia.columns

CategoricalIndex(['Case Fatality Rate (CFR) - \n Confirmed & Probable Cases',
                  'Case Fatality Rate (CFR) - Confirmed & Probable Cases',
                  'Contacts lost to follow-up', 'Contacts seen',
                  'Contacts who completed 21 day \n follow-up',
                  'Contacts who completed 21 day follow-up',
                  'Cumulative (confirmed + probable + suspected)',
                  'Cumulative (confirmed + probable + suspects)',
                  'Cumulative CFR', 'Cumulative admission/isolation',
                  'Cumulative cases among HCW',
                  'Cumulative confirmed, probable and suspected cases',
                  'Cumulative deaths among HCW', 'Currently under follow-up',
                  'New Case/s (Probable)', 'New Case/s (Suspected)',
                  'New admissions', 'New case/s (confirmed)',
                  'Newly Reported Cases in HCW',
                  'Newly Reported deaths in HCW', 'Newly reported contacts',

In [84]:
sl.columns

CategoricalIndex(['cfr', 'contacts_followed', 'contacts_healthy',
                  'contacts_ill', 'contacts_not_seen',
                  'cum_completed_contacts', 'cum_confirmed', 'cum_contacts',
                  'cum_noncase', 'cum_probable', 'cum_suspected',
                  'death_confirmed', 'death_probable', 'death_suspected',
                  'etc_cum_admission', 'etc_cum_deaths', 'etc_cum_discharges',
                  'etc_currently_admitted', 'etc_new_admission',
                  'etc_new_deaths', 'etc_new_discharges', 'negative_corpse',
                  'new_completed_contacts', 'new_confirmed', 'new_contacts',
                  'new_negative', 'new_noncase', 'new_positive',
                  'new_probable', 'new_samples', 'new_suspected', 'pending',
                  'percent_seen', 'population', 'positive_corpse',
                  'repeat_samples', 'total_lab_samples'],
                 categories=['cfr', 'contacts_followed', 'contacts_healthy', 'contacts_ill', 'con

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [85]:
# Setup for location of the dataset of the task 2
MICROBIOME_FOLDER = DATA_FOLDER + "/microbiome"

In [86]:
# Write your answer here
# TODO: Should the data from the spreadsheets be merged in different columns instead of adding indexes as we are doing now
#        Should at least the tissue and stool from the same group of patient be merged?


### Basic analysis of the file formatting:

***
**For the files MIDn.xls with n in [1,9]**

**column 1:**

We see that the first column in the files contain the scientific classification of the microbiomes
Altough it could be kept as a single string, it would have more meaning if splitted
The scientific classification contains the following subdivisions (https://en.wikipedia.org/wiki/Taxonomic_rank): 

       Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species
       
 With this classification, we have a problem: only 6 strings are given in the data set, whilst we have 8 potential divisions in the classification.
 
 After analysis, it seems that Kingdom is not used with bacterias (https://en.wikipedia.org/wiki/Bacteria)
 and always the same as the Phylum in Archea(https://en.wikipedia.org/wiki/Archaea). Therefore, **we will not use Kingdom as a division**.
 
 Also, Species only has a sense in the Eucaryote Domain, for which we have no data, so **we will not use Species as a division**.
 
 In addition, after playing with the data, we found cases where the Family can be named: "Incertae Sedis", which would be classified as the family and genus in our classification. Therefore, we need to check for those cases and re-concatenate the strings to return a proper list in every cases.

In [87]:
# Lets keep those names in a list for further use
scientific_classification = ["domain", "phylum", "class", "order", "family", "genus"]

**Column 2:**

We see that the second column contains an integer value. We can suppose that this value is the number of samples
containing the genus defined by the first column. There's no title to the column, so we don't know yet where
those values came from.

***

**For the file "Metadata.xls"**

**Column 1:**

Titled **"BARCODE"**, give xls file identifier for which the two other columns give more information

**Column 2:**

Titled **"GROUP"**, gives the groups from which each dataset has been sampled. Two informations are contained in those groups, the first is given by "NEC", "Control" or "EXTRACTION CONTROL" and the second is the numbering of the group (which is likely a phase of test), either "1" or "2". We will split those informations in two column, because while using the dataset, we might want to combine all the "NEC" patients or all the patient for a specific phase.

**Column 3:**

Titled **"SAMPLE"**, gives the type of sample that was taken, either tissue, stool or NA. Each group had both types of samples taken. 

***

### Desired formating of the data after analysis

The simple analysis above tells us what are the columns that we will want in our Data Frame

1. **6** column for classification, 1 for each classifier in the "scientific_classification" list. This will caracterise each microbiome individually.

2. **1** column will contain the value associated with each microbiome measurement.

3. **2** columns describing the group of the sample from the metadata. The first column will be called **Group Type** and will contain the "NEC", "Control" or "EXTRACTION CONTROL" value. The second column will be called **Group Phase** and will contain either "1", "2" or "unknow", "unknow" will be used for the "EXCTRACTON CONTROL" group.

4. **1** column describing the type of sample taken from the metadata. This column will be called **Sample** This will be either "tissue", "stool" or "unknow", "unknow" in the case of the EXTRACTION CONTROL group.

For a total of **10** columns


In [88]:
metadata_list = ["group_type", "group_phase", "sample"]

In [89]:
df_col_list = scientific_classification + metadata_list + ["value"]

In [90]:
print(len(df_col_list))
print(df_col_list)

10
['domain', 'phylum', 'class', 'order', 'family', 'genus', 'group_type', 'group_phase', 'sample', 'value']


***
### We want to extract the metadata from the metadata.xls file

The metadata is needed before we start extracting the data from the MB files to create the DataFrame with all the desired columns.


In [91]:
#The actual name of the excel sheet is "Sheet1" and not "Sheet 1" as in the other files.
metadata_raw = pd.read_excel(MICROBIOME_FOLDER+"/metadata.xls", sheetname='Sheet1', header=0)
metadata_raw.columns = metadata_raw.columns.str.lower()


In [92]:
## Extract group phase and group type
group = metadata_raw["group"]
group_type = []
group_phase = []
for index in range(len(group)):                # Index progression must be upward!
    # A special case for the extraction control, we don't want to split it
    if group[index] == "EXTRACTION CONTROL":
        group_type.append(group[index])
        group_phase.append("unknow")
    else:
        splitted_group = group[index].split()
        group_type.append(splitted_group[0])
        group_phase.append(splitted_group[1])
        

In [93]:
metadata_raw['group_type'] = pd.Series(group_type)
metadata_raw['group_phase'] = pd.Series(group_phase)
metadata = metadata_raw.drop("group",  axis=1)
metadata

,barcode,sample,group_type,group_phase
0,MID1,NaN,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


In [94]:
## make sure SAMPLE doesn't contain NaN
metadata["sample"].fillna(value="unknow", inplace=True)
metadata

,barcode,sample,group_type,group_phase
0,MID1,unknow,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


The format of the metadata is now as we want it

***
### We now want to extract the data from the datasheets

** We create a function to extract the classifications **

We want to extract the classifications from the first column of the datasheets. We will need to do string parsing. Regex are a great tool to remove the unwanted characters from the format. The only unwanted characters are the double quotes ". Also, as said before, we need to manage the case where we have the name "Incertae Sedis" as a Family (4th position)

In [95]:
import re

# TODO: this could be made more general, for every potential cases where there's more that 6 elements
# Next function replace " in the classifier_string, split the string to have an indexable list and return it
def get_classifiers(classifier_string):
    classified_list = re.subn("\"", "", classifier_string)[0].split()
    
    # Special case management
    if len(classified_list) > 6:
        # We join the first extra location with the family
        classified_list[4] = classified_list[4] + "_" + classified_list[5]
        del classified_list[5]
        # Use a loop to allow for harbitrary number of words in the genus and for safety
        while len(classified_list) > 6:
            classified_list[5] += "_" + classified_list[6]
            del classified_list[6]
        
    return classified_list
    # Check if the 4th value is "Incertae"
#    if "Incertae" in classified_list:
#        classified_list[4] = classified_list[4] + "_" + classified_list[5]
#        if classified_list[7]:
#            classified_list[5] = classified_list[6] + "_" + classified_list[7]
#            classified_list.pop()
#            classified_list.pop()
#        else:
#            classified_list[5] = classified_list[6]
#            classified_list.pop()
#    return classified_list

** We can now attempt to create the desired dataframe **

In [96]:
# In this section, we test the analysis of a single datasheet

test_data = pd.read_excel(MICROBIOME_FOLDER+"/MID1.xls", sheetname='Sheet 1', header=None)
test_data.columns = ["raw_classification", "value"]
classifier_df = pd.DataFrame(columns=df_col_list)
classifier_array = []
for row_data in test_data.itertuples():
    classifier_array.append(get_classifiers(row_data.raw_classification))
classifier_series = pd.DataFrame(classifier_array, columns=scientific_classification)
classifier_series["value"] = test_data["value"]
classifier_series

,domain,phylum,class,order,family,genus,value
0,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Desulfurococcaceae,Ignisphaera,7
1,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,2
2,Archaea,Crenarchaeota,Thermoprotei,Sulfolobales,Sulfolobaceae,Stygiolobus,3
3,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,Thermofilaceae,Thermofilum,3
4,Archaea,Euryarchaeota,Methanomicrobia,Methanocellales,Methanocellaceae,Methanocella,7
5,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methanosarcinaceae,Methanimicrococcus,1
6,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methermicoccaceae,Methermicoccus,1
7,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Ferroglobus,1
8,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Geoglobus,1
9,Archaea,Euryarchaeota,Halobacteria,Halobacteriales,Halobacteriaceae,Haloplanus,1


In [97]:
# Loop over all the datasheets
clean_data = pd.DataFrame(columns=df_col_list)
for metadata_row in metadata.itertuples():
    raw_data = pd.read_excel(MICROBIOME_FOLDER+"/"+metadata_row.barcode+".xls", sheetname='Sheet 1', header=None)
    # Change column names to something clearer
    raw_data.columns = ["raw_classification", "value"]
    
    # For each datasheet create a local classified set of data
    classifier_array = []
    for row_data in raw_data.itertuples():
        classifier_array.append(get_classifiers(row_data.raw_classification))
    local_classified = pd.DataFrame(classifier_array, columns=scientific_classification)
    # To the local set of data add the columns that are not the classification
    local_classified["value"]       = test_data["value"]
    local_classified["group_type"]  = metadata_row.group_type
    local_classified["group_phase"] = metadata_row.group_phase
    local_classified["sample"]      = metadata_row.sample
    # Append this local set of data to "clean_data", which is the merge of the data from each measurements
    clean_data = clean_data.append(local_classified, ignore_index=True)

clean_data

,class,domain,family,genus,group_phase,group_type,order,phylum,sample,value
0,Thermoprotei,Archaea,Desulfurococcaceae,Ignisphaera,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,7
1,Thermoprotei,Archaea,Pyrodictiaceae,Pyrolobus,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,2
2,Thermoprotei,Archaea,Sulfolobaceae,Stygiolobus,unknow,EXTRACTION CONTROL,Sulfolobales,Crenarchaeota,unknow,3
3,Thermoprotei,Archaea,Thermofilaceae,Thermofilum,unknow,EXTRACTION CONTROL,Thermoproteales,Crenarchaeota,unknow,3
4,Methanomicrobia,Archaea,Methanocellaceae,Methanocella,unknow,EXTRACTION CONTROL,Methanocellales,Euryarchaeota,unknow,7
5,Methanomicrobia,Archaea,Methanosarcinaceae,Methanimicrococcus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
6,Methanomicrobia,Archaea,Methermicoccaceae,Methermicoccus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
7,Archaeoglobi,Archaea,Archaeoglobaceae,Ferroglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
8,Archaeoglobi,Archaea,Archaeoglobaceae,Geoglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
9,Halobacteria,Archaea,Halobacteriaceae,Haloplanus,unknow,EXTRACTION CONTROL,Halobacteriales,Euryarchaeota,unknow,1


** Now lets replace all the None values by unknow **

In [98]:
for item in clean_data.iteritems():
    item[1].fillna(value="unknow", inplace=True)

In [99]:
clean_data.head(10)

,class,domain,family,genus,group_phase,group_type,order,phylum,sample,value
0,Thermoprotei,Archaea,Desulfurococcaceae,Ignisphaera,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,7
1,Thermoprotei,Archaea,Pyrodictiaceae,Pyrolobus,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,2
2,Thermoprotei,Archaea,Sulfolobaceae,Stygiolobus,unknow,EXTRACTION CONTROL,Sulfolobales,Crenarchaeota,unknow,3
3,Thermoprotei,Archaea,Thermofilaceae,Thermofilum,unknow,EXTRACTION CONTROL,Thermoproteales,Crenarchaeota,unknow,3
4,Methanomicrobia,Archaea,Methanocellaceae,Methanocella,unknow,EXTRACTION CONTROL,Methanocellales,Euryarchaeota,unknow,7
5,Methanomicrobia,Archaea,Methanosarcinaceae,Methanimicrococcus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
6,Methanomicrobia,Archaea,Methermicoccaceae,Methermicoccus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
7,Archaeoglobi,Archaea,Archaeoglobaceae,Ferroglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
8,Archaeoglobi,Archaea,Archaeoglobaceae,Geoglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
9,Halobacteria,Archaea,Halobacteriaceae,Haloplanus,unknow,EXTRACTION CONTROL,Halobacteriales,Euryarchaeota,unknow,1


** Now lets manage the index of the DataFrame **

For now, we consider that every column but the value can be considered as a metadata. 

In [112]:
indexed_data = clean_data.set_index(list(metadata_list + scientific_classification))
indexed_data.head(10)

value
group_type         group_phase sample domain  phylum        class           order             family             genus                   
EXTRACTION CONTROL unknow      unknow Archaea Crenarchaeota Thermoprotei    Desulfurococcales Desulfurococcaceae Ignisphaera            7
                                                                                              Pyrodictiaceae     Pyrolobus              2
                                                                            Sulfolobales      Sulfolobaceae      Stygiolobus            3
                                                                            Thermoproteales   Thermofilaceae     Thermofilum            3
                                              Euryarchaeota Methanomicrobia Methanocellales   Methanocellaceae   Methanocella           7
                                                                            Methanosarcinales Methanosarcinaceae Methanimicrococcus     1
                                                                                              Methermicoccaceae  Methermicoccus         1
                                                            Archaeoglobi    Archaeoglobales   Archaeoglobaceae   Ferroglobus            1
                                                                                                                 Geoglobus              1
                                                            Halobacteria    Halobacteriales   Halobacteriaceae   Haloplanus             1

In [113]:
indexed_data.index

MultiIndex(levels=[['Control', 'EXTRACTION CONTROL', 'NEC'], ['1', '2', 'unknow'], ['stool', 'tissue', 'unknow'], ['Archaea', 'Bacteria'], ['Acidobacteria', 'Actinobacteria', 'Aquificae', 'Bacteroidetes', 'Caldiserica', 'Chlamydiae', 'Chlorobi', 'Chloroflexi', 'Chrysiogenetes', 'Crenarchaeota', 'Cyanobacteria', 'Deferribacteres', 'Deinococcus-Thermus', 'Euryarchaeota', 'Firmicutes', 'Fusobacteria', 'Gemmatimonadetes', 'Lentisphaerae', 'Nitrospira', 'OD1', 'OP11', 'Planctomycetes', 'Proteobacteria', 'Spirochaetes', 'Synergistetes', 'TM7', 'Tenericutes', 'Thermodesulfobacteria', 'Thermotogae', 'Verrucomicrobia'], ['Acidobacteria_Gp16', 'Actinobacteria', 'Alphaproteobacteria', 'Anaerolineae', 'Aquificae', 'Archaeoglobi', 'Bacilli', 'Bacteroidetes_incertae_sedis', 'Bacteroidia', 'Betaproteobacteria', 'Caldilineae', 'Caldisericia', 'Chlamydiae', 'Chlorobia', 'Chloroflexi', 'Chrysiogenetes', 'Clostridia', 'Cyanobacteria', 'Deferribacteres', 'Dehalococcoidetes', 'Deinococci', 'Deltaproteobact

In [115]:
indexed_data.index.is_unique

True

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [100]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [101]:
# Write your answer here